<h3><b>PROBLEM STATEMENT:</h3></b>
<p style="font-size:15px">    
This business case is on inventory management. Keeping inventory of spares in various service centres to meet market demand remains a challenge as most service centres spend significant amount on spare parts inventory costs. Inspite of this, the availability of spare parts to meet customers future demands remains a pertinent problem yet to be resolved by service centers.
</p>

<h3><b>PROJECT OBJECTIVE:</h3></b>
<p style="font-size:15px">    
The objective of this project is to simply design and build a predictive modeling solution that can forecast the future demand quantity of different sparepart categories for a service center so they can can begin to meet Just-In-Time(JIT) standards.
</p>

<h3><b>PROJECT SUMMARY:</h3></b>
<p style="font-size:15px">
This project focuses on developing a predictive model to forecast spare parts demand for a service center, supporting Just-In-Time (JIT) inventory management practices. Using historical transaction and service data, I engineered key features that capture customer behavior patterns, time-based demand cycles, and operational delays. The predictive model designed for this project aims to accurately estimate the demand quantity of spare parts in the future, thereby enabling the service center client to optimize inventory levels, reduce holding costs, prevent stockouts, and enhance customer satisfaction. By leveraging data-driven inventory forecasting the service center can significantly improve operational efficiency and align with modern JIT inventory standards.
</p>

In [473]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 587.4 kB/s eta 0:00:0000:0100:01


In [475]:
import pandas as pd
import numpy as np
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder,OrdinalEncoder
import warnings
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import pickle
from sklearn.pipeline import Pipeline
%matplotlib inline
warnings.filterwarnings('ignore')

In [378]:
df = pd.read_excel('Spares_Client_Inventory_Book1.xlsx')
df

,Invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,lag_days,jobcard_day_of_week,jobcard_month,jobcard_year,vehicle_usage_tier,demand_quantity
0,2017-05-30,2017-05-30,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL,0,Tuesday,May,2017,Medium,1
1,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL,2,Wednesday,May,2017,Low,6
2,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH,2,Wednesday,May,2017,Low,1
3,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES,2,Wednesday,May,2017,Low,1
4,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL,2,Wednesday,May,2017,Low,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28076,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER,0,Sunday,January,2019,Low,1
28077,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE,0,Sunday,January,2019,Low,1
28078,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER,0,Sunday,January,2019,Low,1
28079,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL,0,Sunday,January,2019,Low,2


In [379]:
df.shape

(28081, 13)

In [380]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28081 entries, 0 to 28080
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Invoice_date           28081 non-null  datetime64[ns]
 1   job_card_date          28081 non-null  datetime64[ns]
 2   business_partner_name  28081 non-null  object        
 3   vehicle_no             28081 non-null  object        
 4   vehicle_model          28081 non-null  object        
 5   current_km_reading     28081 non-null  int64         
 6   invoice_line_text      28081 non-null  object        
 7   lag_days               28081 non-null  int64         
 8   jobcard_day_of_week    28081 non-null  object        
 9   jobcard_month          28081 non-null  object        
 10  jobcard_year           28081 non-null  int64         
 11  vehicle_usage_tier     28081 non-null  object        
 12  demand_quantity        28081 non-null  int64         
dtypes

In [381]:
df.isnull().sum()

Invoice_date             0
job_card_date            0
business_partner_name    0
vehicle_no               0
vehicle_model            0
current_km_reading       0
invoice_line_text        0
lag_days                 0
jobcard_day_of_week      0
jobcard_month            0
jobcard_year             0
vehicle_usage_tier       0
demand_quantity          0
dtype: int64

In [382]:
df = df.dropna()

In [383]:
df.isnull().sum()

Invoice_date             0
job_card_date            0
business_partner_name    0
vehicle_no               0
vehicle_model            0
current_km_reading       0
invoice_line_text        0
lag_days                 0
jobcard_day_of_week      0
jobcard_month            0
jobcard_year             0
vehicle_usage_tier       0
demand_quantity          0
dtype: int64

In [384]:
df.duplicated().sum()

0

In [385]:
df = df.drop_duplicates()

In [386]:
df.isnull().sum()

Invoice_date             0
job_card_date            0
business_partner_name    0
vehicle_no               0
vehicle_model            0
current_km_reading       0
invoice_line_text        0
lag_days                 0
jobcard_day_of_week      0
jobcard_month            0
jobcard_year             0
vehicle_usage_tier       0
demand_quantity          0
dtype: int64

In [387]:
#Converted['Invoice_date'] and ['job_card_date'] from object dtype to datetime dtype.

df['Invoice_date'] = pd.to_datetime(df['Invoice_date'])
df['job_card_date'] = pd.to_datetime(df['job_card_date'])

In [388]:
df

,Invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,lag_days,jobcard_day_of_week,jobcard_month,jobcard_year,vehicle_usage_tier,demand_quantity
0,2017-05-30,2017-05-30,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL,0,Tuesday,May,2017,Medium,1
1,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL,2,Wednesday,May,2017,Low,6
2,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH,2,Wednesday,May,2017,Low,1
3,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES,2,Wednesday,May,2017,Low,1
4,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL,2,Wednesday,May,2017,Low,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28076,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER,0,Sunday,January,2019,Low,1
28077,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE,0,Sunday,January,2019,Low,1
28078,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER,0,Sunday,January,2019,Low,1
28079,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL,0,Sunday,January,2019,Low,2


In [389]:
df.shape

(28081, 13)

In [390]:
#'lag_days': Number of days between when service was requested and when invoice was generated.

df['lag_days'] = (df['Invoice_date'] - df['job_card_date']).dt.days

In [391]:
#'jobcard_dayofweek': Day of the week when service order was placed (e.g., Monday, Tuesday).

df['jobcard_day_of_week'] = df['job_card_date'].dt.day_name()

In [392]:
#'jobcard_month': Month when service order was placed (e.g., January, February).

df['jobcard_month'] = df['job_card_date'].dt.month_name()

In [393]:
#'jobcard_year': Year when service order was placed.

df['jobcard_year'] = df['job_card_date'].dt.year

In [394]:

def categorize_mileage(mileage):
    
    if pd.isnull(mileage):
        return "Unknown"
    elif mileage < 20000:
        return "Low"
    elif mileage < 80000:
        return "Medium"
    else:
        return "High"


In [395]:
#'vehicle_usage_tier': Categorized vehicle mileage into Low, Medium, High.

df['vehicle_usage_tier'] = df['current_km_reading'].apply(categorize_mileage)

In [396]:
# 'demand_quantity': Set constant demand quantity as 1 for each invoiced spare part transaction.

df['demand_quantity'] = df.groupby(['invoice_line_text', 'job_card_date'])['invoice_line_text'].transform('count')

In [397]:
pd.set_option('display.max_rows',None)

In [398]:
pd.reset_option('display.max_rows',None)

In [399]:
df

,Invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,lag_days,jobcard_day_of_week,jobcard_month,jobcard_year,vehicle_usage_tier,demand_quantity
0,2017-05-30,2017-05-30,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL,0,Tuesday,May,2017,Medium,1
1,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL,2,Wednesday,May,2017,Low,6
2,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH,2,Wednesday,May,2017,Low,1
3,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES,2,Wednesday,May,2017,Low,1
4,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL,2,Wednesday,May,2017,Low,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28076,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER,0,Sunday,January,2019,Low,1
28077,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE,0,Sunday,January,2019,Low,1
28078,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER,0,Sunday,January,2019,Low,1
28079,2019-01-06,2019-01-06,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL,0,Sunday,January,2019,Low,2


In [400]:
df.describe()

,Invoice_date,job_card_date,current_km_reading,lag_days,jobcard_year,demand_quantity
count,28081,28081,28081.000000,28081.000000,28081.000000,28081.000000
mean,2018-04-20 21:46:37.208076544,2018-04-20 20:56:43.468537088,19335.123037,0.034650,2017.709519,4.211638
min,2017-05-30 00:00:00,2017-05-30 00:00:00,0.000000,-1.000000,2017.000000,1.000000
25%,2017-11-28 00:00:00,2017-11-28 00:00:00,3959.000000,0.000000,2017.000000,1.000000
50%,2018-05-10 00:00:00,2018-05-10 00:00:00,12347.000000,0.000000,2018.000000,3.000000
75%,2018-09-21 00:00:00,2018-09-21 00:00:00,27906.000000,0.000000,2018.000000,6.000000
max,2019-01-06 00:00:00,2019-01-06 00:00:00,610112.000000,19.000000,2019.000000,17.000000
std,NaN,NaN,25301.781847,0.441575,0.480071,3.184124


In [401]:
df.describe(include='object')

,business_partner_name,vehicle_no,vehicle_model,invoice_line_text,jobcard_day_of_week,jobcard_month,vehicle_usage_tier
count,28081,28081,28081,28081,28081,28081,28081
unique,1010,846,28,502,7,12,3
top,venkXXXXXXXXXX,KA53EVXXXX,BAJAJ PULSAR 150,ENGINE OIL,Saturday,December,Low
freq,416,1299,8498,3780,5582,3584,17829


In [402]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28081 entries, 0 to 28080
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Invoice_date           28081 non-null  datetime64[ns]
 1   job_card_date          28081 non-null  datetime64[ns]
 2   business_partner_name  28081 non-null  object        
 3   vehicle_no             28081 non-null  object        
 4   vehicle_model          28081 non-null  object        
 5   current_km_reading     28081 non-null  int64         
 6   invoice_line_text      28081 non-null  object        
 7   lag_days               28081 non-null  int64         
 8   jobcard_day_of_week    28081 non-null  object        
 9   jobcard_month          28081 non-null  object        
 10  jobcard_year           28081 non-null  int32         
 11  vehicle_usage_tier     28081 non-null  object        
 12  demand_quantity        28081 non-null  int64         
dtypes

In [403]:
df.to_excel('Spares_Client_Inventory_Book1.xlsx', index=False)


In [404]:
# Dropped irrelevant columns and old columns that have been replaced with new features via feature extraction.

columns_dropped = ['Invoice_date', 'job_card_date', 'business_partner_name', 
                   'vehicle_no',] 

df.drop(columns=['Invoice_date', 'job_card_date', 'business_partner_name', 'vehicle_no',], inplace=True)

In [405]:
df

,vehicle_model,current_km_reading,invoice_line_text,lag_days,jobcard_day_of_week,jobcard_month,jobcard_year,vehicle_usage_tier,demand_quantity
0,BAJAJ AVENGER STREET 220,50000,ENGINE OIL,0,Tuesday,May,2017,Medium,1
1,BAJAJ PULSAR NS 200,758,ENGINE OIL,2,Wednesday,May,2017,Low,6
2,BAJAJ PULSAR NS 200,758,POLISH,2,Wednesday,May,2017,Low,1
3,BAJAJ PULSAR NS 200,758,CONSUMABLES,2,Wednesday,May,2017,Low,1
4,BAJAJ PULSAR NS 200,758,COOLANT OIL,2,Wednesday,May,2017,Low,1
...,...,...,...,...,...,...,...,...,...
28076,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER,0,Sunday,January,2019,Low,1
28077,BAJAJ DISCOVER 125,12805,CLUTCH CABLE,0,Sunday,January,2019,Low,1
28078,BAJAJ DISCOVER 125,12805,OIL FILTER,0,Sunday,January,2019,Low,1
28079,BAJAJ DISCOVER 125,12805,DISC OIL,0,Sunday,January,2019,Low,2


In [406]:
X = df.drop('demand_quantity', axis=1)  #Defined the Input data
y = df['demand_quantity']              # Defined the Target data

In [407]:
from sklearn.preprocessing import StandardScaler

# List of numerical features
numerical_features = ['current_km_reading', 'lag_days', 'jobcard_year']

# Initialize scaler
scaler = StandardScaler()

# Apply scaling
df[numerical_features] = scaler.fit_transform(df[numerical_features])


In [408]:
df

,vehicle_model,current_km_reading,invoice_line_text,lag_days,jobcard_day_of_week,jobcard_month,jobcard_year,vehicle_usage_tier,demand_quantity
0,BAJAJ AVENGER STREET 220,1.211987,ENGINE OIL,-0.078470,Tuesday,May,-1.477972,Medium,1
1,BAJAJ PULSAR NS 200,-0.734235,ENGINE OIL,4.450849,Wednesday,May,-1.477972,Low,6
2,BAJAJ PULSAR NS 200,-0.734235,POLISH,4.450849,Wednesday,May,-1.477972,Low,1
3,BAJAJ PULSAR NS 200,-0.734235,CONSUMABLES,4.450849,Wednesday,May,-1.477972,Low,1
4,BAJAJ PULSAR NS 200,-0.734235,COOLANT OIL,4.450849,Wednesday,May,-1.477972,Low,1
...,...,...,...,...,...,...,...,...,...
28076,BAJAJ DISCOVER 125,-0.258094,SPROCKET RUBBER,-0.078470,Sunday,January,2.688153,Low,1
28077,BAJAJ DISCOVER 125,-0.258094,CLUTCH CABLE,-0.078470,Sunday,January,2.688153,Low,1
28078,BAJAJ DISCOVER 125,-0.258094,OIL FILTER,-0.078470,Sunday,January,2.688153,Low,1
28079,BAJAJ DISCOVER 125,-0.258094,DISC OIL,-0.078470,Sunday,January,2.688153,Low,2


In [409]:
# List of categorical features.

categorical_features = ['vehicle_model', 'invoice_line_text', 'jobcard_day_of_week', 'jobcard_month', 'vehicle_usage_tier']

# One-hot encoding.

df = pd.get_dummies(df, columns=categorical_features, dtype='int', drop_first=True)

In [410]:
df

,current_km_reading,lag_days,jobcard_year,demand_quantity,vehicle_model_BAJAJ AVENGER STREET 180,vehicle_model_BAJAJ AVENGER STREET 220,vehicle_model_BAJAJ BOXER CT,vehicle_model_BAJAJ CALIBER,vehicle_model_BAJAJ CT 100,vehicle_model_BAJAJ DISCOVER,vehicle_model_BAJAJ DISCOVER 125,vehicle_model_BAJAJ DISCOVER 135,vehicle_model_BAJAJ DISCOVER 150,vehicle_model_BAJAJ DOMINAR 400 ABS,vehicle_model_BAJAJ PLATINA,vehicle_model_BAJAJ PULSAR 150,vehicle_model_BAJAJ PULSAR 180,vehicle_model_BAJAJ PULSAR 220,vehicle_model_BAJAJ PULSAR AS 200,vehicle_model_BAJAJ PULSAR LS135,vehicle_model_BAJAJ PULSAR NS 160,vehicle_model_BAJAJ PULSAR NS 200,vehicle_model_BAJAJ PULSAR RS 200,vehicle_model_BAJAJ V,vehicle_model_BAJAJ V125,vehicle_model_BAJAJ V15,vehicle_model_BAJAJ V150,vehicle_model_BAJAJ WIND 125,vehicle_model_BAJAJ XCD 125,vehicle_model_BAJAJ XCD 135,vehicle_model_WAVE,invoice_line_text_12 SIZE BOLT,invoice_line_text_3M OIL,invoice_line_text_4 WAY SWITCH,invoice_line_text_6004 BEARING,invoice_line_text_6202 BEARING,invoice_line_text_6303 BEARING,invoice_line_text_8 SIZE BOLT,invoice_line_text_ACCELATOR CABLE,invoice_line_text_ADD ON FEES,invoice_line_text_AIR FILTER,invoice_line_text_AIR FILTER CHECKUP,invoice_line_text_AIR INTAKE,invoice_line_text_AXLE,invoice_line_text_BALANCING GEAR,invoice_line_text_BATTERY,invoice_line_text_BATTERY BELT,invoice_line_text_BATTERY CHARGE,invoice_line_text_BATTERY CLIP,invoice_line_text_BATTERY TARMINAL,invoice_line_text_BC UNIT,invoice_line_text_BEARING,invoice_line_text_BEARING SET,invoice_line_text_BEARING WIELDING,invoice_line_text_BEDDING BOLT,invoice_line_text_BODY,invoice_line_text_BODY COVER,invoice_line_text_BOLT REMOVAL,invoice_line_text_BOLT WIELDING,invoice_line_text_BORE,invoice_line_text_BORE GASKET,invoice_line_text_BORE KIT,invoice_line_text_BOTTLE COIL,invoice_line_text_BRAKE ADJUSTMENT,invoice_line_text_BRAKE BUSH KIT,invoice_line_text_BRAKE CABLE,invoice_line_text_BRAKE LEVER,invoice_line_text_BRAKE LIGHT SPRING,invoice_line_text_BRAKE LIGHT SWITCH,invoice_line_text_BRAKE LIGHT SWITCH,invoice_line_text_BRAKE PEDAL,invoice_line_text_BRAKE PEDAL RUBBER,invoice_line_text_BRAKE PEDAL SPRING,invoice_line_text_BRAKE ROD,invoice_line_text_BRAKE ROD SPRING,invoice_line_text_BRAKE RUBBER,invoice_line_text_BRAKE SHOE,invoice_line_text_BRAKE SWITCH,invoice_line_text_BULB,invoice_line_text_BULB,invoice_line_text_BULB HOLDER,invoice_line_text_BUSH KIT,invoice_line_text_BUZZER,invoice_line_text_CABLE LOCK,invoice_line_text_CAHIN PAD SCREW,invoice_line_text_CALIPER ASSEMBLY,invoice_line_text_CALIPER RUBBER KIT,invoice_line_text_CAM SHAFT,invoice_line_text_CAM SHAFT SPRING,invoice_line_text_CARBURETOR,invoice_line_text_CARBURETOR ASSEMBLY,invoice_line_text_CARBURETOR CLEANING,invoice_line_text_CARBURETOR DIAPHRAGM RUBBER,invoice_line_text_CARBURETOR PLATE,invoice_line_text_CARBURETOR SPRING,invoice_line_text_CASE BEARING,invoice_line_text_CASE WELDING,invoice_line_text_CDI UNIT,invoice_line_text_CENTER STAND,invoice_line_text_CENTER STAND SPRING,invoice_line_text_CHAIN ADJUSTER,invoice_line_text_CHAIN ADJUSTER PLATE,invoice_line_text_CHAIN COVER BEADING,invoice_line_text_CHAIN COVER BOLT,invoice_line_text_CHAIN COVER RUBBER,invoice_line_text_CHAIN LINK LOCK,invoice_line_text_CHAIN LUBRICATION,invoice_line_text_CHAIN OVERHAUL,invoice_line_text_CHAIN PAD,invoice_line_text_CHAIN SET,invoice_line_text_CHAIN SPROCKET,invoice_line_text_CHOKE CABLE,invoice_line_text_CHOKE SENSOR,invoice_line_text_CLUTCH ADJUSTMENT,invoice_line_text_CLUTCH ASSEMBLY,invoice_line_text_CLUTCH BELL REST,invoice_line_text_CLUTCH CABLE,invoice_line_text_CLUTCH CAM,invoice_line_text_CLUTCH CAM SPRING,invoice_line_text_CLUTCH CAM SWITCH,invoice_line_text_CLUTCH COVER GASKET,invoice_line_text_CLUTCH COVER PACKING,invoice_line_text_CLUTCH LEVER,invoice_line_text_CLUTCH OUTLET PIPE,invoice_line_text_CLUTCH OVERHUAL,invoice_line_text_CLUTCH REPALCEMENT,invoice_line_text_CLUTCH ROLLERS,invoice_line_text_CLUTCH SHOE,invoice_line_text_

In [447]:
y.head()

0    1
1    6
2    1
3    1
4    1
Name: demand_quantity, dtype: int64

In [449]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Split the data into train and test sets.

X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [511]:
#MODEL BUILDING

import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import numpy as np

In [479]:
# 2. Initialize the XGBoost Regressor.

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)

In [481]:
# 3. Fit the model to the training data.

xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [483]:
# 4. Initialize predictions on the X_test data.

y_pred_xgb = xgb_model.predict(X_test)

In [515]:
# 5. Evaluate the model.

r2_xgb = r2_score(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)

print("XGBoost Regressor Performance:")
print(f"R² Score: {r2_xgb:.4f}")
print(f"Mean Squared Error: {mse_xgb:.2f}")

XGBoost Regressor Performance:
R² Score: 0.7377
Mean Squared Error: 2.76


In [501]:
#HYPER-PARAMETER TUNING OF XGB MODEL

# 6. Defined the parameter grid.
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

In [503]:
# 7. Initializing XGB model.

xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)


In [505]:
# 8. GridSearchCV setup.

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    verbose=1,
    n_jobs=-1
)

In [517]:
# 9. Fit GridSearchCV parameters to training data.

grid_search.fit(X_train, y_train)

# 10. Best parameters and score.

print("Best Parameters:", grid_search.best_params_)
print("Best R² Score on CV Set:", grid_search.best_score_)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Parameters: {'colsample_bytree': 1, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}
Best R² Score on CV Set: 0.754322510241705


In [519]:
# 11. Predict with best estimator.

best_xgb = grid_search.best_estimator_
y_pred_tuned = best_xgb.predict(X_test)

In [525]:
# 12. Evaluate the tuned model.

r2 = r2_score(y_test, y_pred_tuned)
mse = mean_squared_error(y_test, y_pred_tuned)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_tuned)

print("Tuned XGBoost Model Performance:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

Tuned XGBoost Model Performance:
R² Score: 0.7876
RMSE: 1.49
MAE: 1.09


<h3><b>MODEL REPORT SUMMARY – Spares Inventory Forecasting Project</h3></b>
<p style="font-size:15px">    
For company xyz to accurately forecast spare parts demand and support Just-In-Time (JIT) inventory planning, multiple models were implemented and evaluated. These included a <b>Linear Regression model</b> as a baseline, a <b>Random Forest Regressor</b> to capture complex feature interactions, and an <b>XGBoost Regressor</b> for its efficiency and accuracy in handling structured data. Among these models implemented, XGBoost delivered the best performance based on RMSE and R² score, making it the most preferred model for deployment.
</p>